In [1]:
import yfinance as yf
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas_ta as ta
import numpy as np
import pandas as pd
from yahoo_fin import stock_info as si
from yahooquery import Ticker

def import_data(stock, startDate):
    data = yf.download(stock, start=startDate, interval='1d')
    # add stochastic oscillator to data
    stoch = ta.stoch(data['High'], data['Low'], data['Close'])
    data = data.join(stoch)

    data.reset_index(inplace=True)
    return data


In [2]:
def findPivots(data, window):
    data['isPivot'] = 0
    for curr in range(window, len(data) - window):
        is_high = True
        is_low = True
        for i in range(curr - window, curr + window + 1):
            if data.loc[curr, 'High'] < data.loc[i, 'High']:
                is_high = False
            if data.loc[curr, 'Low'] > data.loc[i, 'Low']:
                is_low = False

        if is_high:
            data.loc[curr, 'isPivot'] = 1
        elif is_low:
            data.loc[curr, 'isPivot'] = -1

In [3]:
def printCandlesticks(data):
    # data = data['2020-01-01':]
    
    fig = go.Figure(data=[go.Candlestick(x=data.Date,
                open=data['Open'],
                high=data['High'],
                low=data['Low'],
                close=data['Close'])])
        
    pivot_highs = data[data['isPivot'] == 1]
    pivot_lows = data[data['isPivot'] == -1]   

    fig.add_scatter(x=pivot_highs.Date, y=pivot_highs['High'], mode='markers', name='Pivot Highs', marker_color='purple')
    fig.add_scatter(x=pivot_lows.Date, y=pivot_lows['Low'], mode='markers', name='Pivot Lows', marker_color='purple')

    support_highs = data.loc[data['hasSupport'] == 1]
    support_lows = data.loc[data['hasSupport'] == -1]

    fig.add_scatter(x=support_highs.Date, y=support_highs['High'], mode='markers', name='Support Highs', marker_color='orange')
    fig.add_scatter(x=support_lows.Date, y=support_lows['Low'], mode='markers', name='Support Lows', marker_color='orange')

    breakout_highs = data.loc[data['isBreakout'] == 1].Date
    breakout_lows = data.loc[data['isBreakout'] == -1].Date

    for date in breakout_highs:
        fig.add_vline(x=date, line_width=1, line_dash="dash", line_color="green")
    for date in breakout_lows:
        fig.add_vline(x=date, line_width=1, line_dash="dash", line_color="red")

    fig.update_layout(height=800) 
    
    fig.show()

In [4]:
# window must be higher than pivot window
def findBreakouts(data, backCandles, window, tolerance):
    data['isBreakout'] = 0
    data['hasSupport'] = 0

    for curr in range(backCandles, len(data)):
        temp = data.iloc[curr - backCandles - window: curr - window]

        lows = temp[temp['isPivot'] == -1].tail(3)
        highs = temp[temp['isPivot'] == 1].tail(3)
        
        if len(lows) == 3:
            hasSupport = True
            avgLow = lows['Low'].mean()
            for low in lows['Low']:
                if abs(low - avgLow) > (avgLow*0.05):
                    hasSupport = False
                    break

            if hasSupport:
                if data.loc[curr, 'isPivot'] == -1:
                    data.loc[curr, 'hasSupport'] = -1
                support_low = lows['Low'].min() * (1 - tolerance)
                if data.loc[curr, 'Low'] < support_low:
                    data.loc[curr, 'isBreakout'] = -1

        if len(highs) == 3:
            hasResistance = True
            avgHigh = highs['High'].mean()
            for high in highs['High']:
                if abs(high - avgHigh) > (avgHigh*0.05):
                    hasResistance = False
                    break

            if hasResistance:
                if data.loc[curr, 'isPivot'] == 1:
                    data.loc[curr, 'hasSupport'] = 1
                resistance_high = highs['High'].max() * (1 + tolerance)
                if data.loc[curr, 'High'] > resistance_high:
                    data.loc[curr, 'isBreakout'] = 1

In [5]:
def filter_equity_tickers(tickers):
    equity_tickers = []
    for ticker in tickers:
        try:
            stock = yf.Ticker(ticker)
            type = stock.info['quoteType']   
            if type == 'EQUITY':
                equity_tickers.append(ticker)     
        except Exception as e:
            print(f"Could not process ticker {ticker}: {e}")
    return equity_tickers

In [93]:
data = import_data('MSSAU', '2024-01-01')
findPivots(data, 5)
findBreakouts(data, 100, 6, .001)
printCandlesticks(data)

[*********************100%%**********************]  1 of 1 completed


In [6]:
# tickers = si.tickers_sp500() + si.tickers_nasdaq() + si.tickers_dow()
# tickers = list(set(tickers))

# stocks = filter_equity_tickers(tickers)

# print(len(stocks))
# print(stocks)

tickers = ['VTGN', 'POWI', 'RDIB', 'PMTS', 'TSCO', 'ACCD', 'LCNB', 'SHC', 'LILM', 'RAVE', 'ACHL', 'ZLAB', 'ASMB', 'BA', 'KIDS', 'YY', 'RCMT', 'ACIU', 'RCKT', 'ATLCL', 'EM', 'ADNWW', 'KRMD', 'LDTCW', 'IGIC', 'TZOO', 'OXLCO', 'OST', 'COMM', 'YGMZ', 'BYNO', 'DH', 'FRZA', 'FORR', 'HTOOW', 'RIOT', 'EA', 'LSTA', 'DGHI', 'SQFTW', 'SOHU', 'CACO', 'MSBIP', 'CZFS', 'MHLD', 'LYEL', 'KTOS', 'KEQU', 'NLSP', 'RILYM', 'SVIIW', 'INTEU', 'FLEX', 'IVCP', 'SRRK', 'IROHR', 'ITRI', 'EDSA', 'HOTH', 'KRON', 'AKTX', 'COFS', 'LNZAW', 'NXLIW', 'PALT', 'PNTG', 'PLCE', 'JVA', 'BEATW', 'WTFCP', 'NXTC', 'SSSSL', 'EBC', 'REPL', 'WVVI', 'RNLX', 'PCT', 'ACVA', 'ZCARW', 'ZURA', 'CARM', 'CLMB', 'FRLA', 'REGCO', 'CMCO', 'RVNC', 'GURE', 'CYCCP', 'ANL', 'EGBN', 'EVGRU', 'FEXDW', 'HLVX', 'ANIX', 'AIMDW', 'FEBO', 'SRZN', 'GS', 'PANW', 'GXAI', 'DJTWW', 'CCD', 'CERS', 'FORTY', 'LPLA', 'MRNS', 'OBIO', 'DCOMG', 'XHG', 'CHKEZ', 'MRBK', 'ADP', 'SSTI', 'PLUG', 'FORD', 'SMMT', 'MGRM', 'SGLY', 'INM', 'ALKT', 'MNOV', 'LYRA', 'PFTAW', 'SBRA', 'RDWR', 'BANL', 'HCAT', 'NEOV', 'PRSO', 'PNBK', 'IMAQU', 'MLEC', 'MOB', 'TCBX', 'WIX', 'BIRD', 'GV', 'MTLS', 'ANGHW', 'PETZ', 'PIII', 'AFJKR', 'SUPN', 'RDZNW', 'XOMAO', 'WKHS', 'LBPH', 'ASNS', 'CRWS', 'GILT', 'SFWL', 'FCAP', 'MNMD', 'TBMC', 'BITF', 'PT', 'AKTS', 'PLBC', 'BYRN', 'ACABW', 'IPDN', 'PTGX', 'SAIC', 'TACT', 'DJT', 'ICMB', 'ODD', 'MRCC', 'PAYOW', 'FFIN', 'KPLTW', 'MBIO', 'WFRD', 'WMT', 'PET', 'INTC', 'ZLSWU', 'DTSTW', 'PRPL', 'PEBK', 'PVBC', 'AGNCL', 'QIPT', 'VRTX', 'SLAM', 'STBA', 'CVRX', 'VIRX', 'NOVVR', 'BSY', 'LIFW', 'INOD', 'VMCAU', 'CDROW', 'GODN', 'GFAIW', 'CPBI', 'QH', 'SPKLU', 'ALLT', 'BFRIW', 'EYPT', 'GREE', 'RUSHB', 'CLST', 'ADD', 'TENK', 'TTNP', 'ATAT', 'SMXWW', 'HWC', 'SCWX', 'MACIU', 'BROGW', 'CLRO', 'TRONU', 'SOHOO', 'ODP', 'HBNC', 'NRIX', 'PHIO', 'SPI', 'UBCP', 'AGNCO', 'DLPN', 'GMGI', 'LUNA', 'OPGN', 'SOTK', 'KOSS', 'BOTJ', 'ALHC', 'GORV', 'TOWN', 'AMD', 'RPHM', 'BNIXW', 'LBTYB', 'CERO', 'PPYA', 'AXON', 'ADBE', 'ESCA', 'PDEX', 'VTSI', 'SBCF', 'SCLX', 'AMPH', 'JKHY', 'NRSN', 'BLFY', 'ADVM', 'PMCB', 'STRS', 'AGRI', 'ANNX', 'BHAT', 'GRYP', 'OCEA', 'BNAI', 'BRACR', 'ELYM', 'EYE', 'BOOM', 'GHIXW', 'NHTC', 'GVH', 'NCMI', 'POAI', 'CHEF', 'PRME', 'BDSX', 'HAIAU', 'NVACR', 'ISRG', 'TLSI', 'MURA', 'CODA', 'AAPL', 'LIPO', 'PFX', 'CASH', 'GCMG', 'RAYA', 'OAKUW', 'MACIW', 'REGN', 'UHGWW', 'VSEE', 'XOSWW', 'TPST', 'ISPC', 'IQ', 'CHUY', 'DENN', 'TLGY', 'TCX', 'AIHS', 'DPCS', 'ASTE', 'NLSPW', 'REYN', 'VMD', 'MCHP', 'TIGO', 'HTCR', 'SRBK', 'BELFA', 'LWLG', 'BKYI', 'ALLR', 'RYTM', 'JTAIZ', 'ADMA', 'MMYT', 'TIPT', 'OCSAW', 'WAVSW', 'AISPW', 'VCIG', 'BDTX', 'BELFB', 'FTEK', 'NNAG', 'VBNK', 'BWFG', 'MBCN', 'BAFN', 'CART', 'VEV', 'WORX', 'JEWL', 'EJH', 'OBT', 'SDIG', 'APLD', 'ARQQW', 'VISL', 'GRABW', 'WTFCM', 'SVII', 'AVGO', 'GPCR', 'SHOTW', 'PLUR', 'IDAI', 'BTOC', 'FMNB', 'WAVD', 'UPST', 'ATER', 'UNB', 'UONEK', 'DATS', 'FBYD', 'RAY', 'VUZI', 'RILYT', 'IVCA', 'DYN', 'MOR', 'SGH', 'MVBF', 'MGYR', 'NVVE', 'ME', 'GAME', 'WINVR', 'NNE', 'ELSE', 'IXAQ', 'NTIC', 'RFACR', 'VRME', 'LXEH', 'NTAP', 'IXAQU', 'ZCMD', 'WDC', 'ACGL', 'DHC', 'HUBCW', 'CTSH', 'XAIR', 'RILY', 'MEDS', 'ALNY', 'GCMGW', 'III', 'SYNA', 'SWBI', 'YJ', 'HCM', 'BRY', 'BGLC', 'INTG', 'VINP', 'SIBN', 'JFU', 'ATLC', 'CARE', 'JSM', 'MSAI', 'ELAB', 'CMPX', 'DAIO', 'GOEVW', 'JAGX', 'DGII', 'RELY', 'FBIO', 'HA', 'GIGM', 'ABVE', 'SKYT', 'NEWTZ', 'OPRT', 'ANIK', 'HFBL', 'KALA', 'KNDI', 'SIMAU', 'EVOK', 'XBIT', 'WAVS', 'GLDD', 'XRAY', 'ESHA', 'LSEAW', 'TURN', 'VERB', 'TNON', 'MU', 'VZ', 'ACNB', 'LATGU', 'KWESW', 'FNVTW', 'GP', 'HUDAU', 'CETUU', 'IPGP', 'CCG', 'MAMA', 'MCAAW', 'MLKN', 'SEDG', 'STHO', 'LUXHP', 'BNOX', 'KURA', 'SKYW', 'NIPG', 'SAVA', 'QFIN', 'BWB', 'ELDN', 'PENN', 'DBX', 'ANEB', 'GTI', 'LIVE', 'MANH', 'RBBN', 'GHIXU', 'CSLMU', 'CLPT', 'SABR', 'CTCXW', 'DTCK', 'GDSTU', 'KYTX', 'SOFI', 'PTVE', 'COOP', 'EHGO', 'ONBPP', 'THCPW', 'OLB', 'AIOT', 'CVIIW', 'SDGR', 'ZEOWW', 'CIVB', 'HTBK', 'DNLI', 'BSRR', 'APP', 'HCVI', 'OFS', 'WHLRL', 'CTAS', 'LEXX', 'WLGS', 'HDL', 'SOHON', 'LSEA', 'INTR', 'CSLR', 'SBFMW', 'ATMV', 'HLIT', 'AGBA', 'KRNLU', 'ENTG', 'CALB', 'ATYR', 'ANAB', 'RILYG', 'ALVR', 'CNVS', 'GPRO', 'CTRM', 'TBPH', 'OXBR', 'OMER', 'ON', 'TNDM', 'WULF', 'BEAT', 'NBIX', 'AGNCM', 'POLA', 'NBST', 'TRUE', 'CHCI', 'EZFL', 'SYRS', 'PYPD', 'ASRV', 'AKAM', 'AKRO', 'EVER', 'AMSF', 'CLNN', 'ISSC', 'BTDR', 'INDV', 'INO', 'NCNC', 'LECO', 'ALMS', 'KRNL', 'LIVN', 'FGIWW', 'ABLVW', 'AIMAW', 'HYMCW', 'CBAT', 'ARBK', 'BYU', 'NUTX', 'SSSS', 'MBIN', 'CSLMW', 'MNTK', 'NTRSO', 'TRV', 'ABLLW', 'CRNC', 'ARBEW', 'ZPTA', 'ATNF', 'EXLS', 'ARBKL', 'GDEVW', 'LOTWW', 'GREEL', 'AFMD', 'PRFX', 'SLAMW', 'AKYA', 'SPSC', 'TCRT', 'SLP', 'COEP', 'WAFU', 'ALCYW', 'AXGN', 'UNTY', 'TBLT', 'UKOMW', 'RVSB', 'MITA', 'ASTI', 'HIFS', 'MREO', 'JYD', 'NVMI', 'VICR', 'CCGWW', 'MORN', 'OS', 'KAVL', 'RELI', 'STRA', 'BFC', 'RNW', 'HIVE', 'FLL', 'JTAIW', 'KALU', 'WENA', 'OESX', 'XPEL', 'CLBK', 'SRAD', 'NWL', 'MMSI', 'COOTW', 'ATRA', 'APCX', 'AHG', 'AIEV', 'NXST', 'CMBM', 'APCXW', 'BLBD', 'AURA', 'CFLT', 'MELI', 'FRHC', 'MIGI', 'NUKK', 'DSWL', 'BGC', 'APGE', 'HTIBP', 'MYNZ', 'FARM', 'DIOD', 'NEGG', 'GLTO', 'RGCO', 'PIIIW', 'ABLV', 'BOCNU', 'OCCI', 'ABIO', 'NXT', 'IVCPU', 'KRNY', 'HALO', 'LBRDK', 'EBON', 'UK', 'MDWD', 'QETAU', 'ATLCZ', 'DATSW', 'ABCL', 'OCX', 'ALTO', 'CENTA', 'CELU', 'BNTX', 'CHSCM', 'ENGNW', 'HTLF', 'IDCC', 'BNZIW', 'KRT', 'CFFS', 'SND', 'BNTC', 'INSG', 'TRONW', 'GLRE', 'NFBK', 'NYAX', 'AAON', 'BNRG', 'INVE', 'IMCR', 'ENTO', 'PLPC', 'MRTN', 'SKIN', 'ANSCW', 'CAC', 'TCJH', 'NVNI', 'AKAN', 'UNIT', 'NETD', 'LTRYW', 'CITEU', 'NOTV', 'SHMD', 'AWRE', 'WSC', 'GDHG', 'VERV', 'FLUX', 'MLAB', 'STKL', 'ZAPPW', 'IGTAU', 'DMRC', 'TMUS', 'BOCN', 'CDZIP', 'ORGS', 'ZVZZT', 'ALCO', 'ALAB', 'DOMO', 'WINV', 'CAKE', 'CPIX', 'IBOC', 'OPHC', 'ILAG', 'PLAOW', 'LFMDP', 'BUJAU', 'IVCBW', 'LDWY', 'MYFW', 'WTBA', 'OPBK', 'QNCX', 'ALXO', 'GAMCW', 'MYPS', 'QRHC', 'IROH', 'SYTAW', 'ACB', 'SVCO', 'SNTI', 'UFCS', 'GBBKW', 'IMAQR', 'DECA', 'EVCM', 'INNV', 'BRAC', 'UMBF', 'MACAU', 'INFN', 'HSPO', 'TTD', 'PRTA', 'CXAIW', 'NNAGR', 'BNZI', 'PODC', 'IPWR', 'ALIM', 'EWBC', 'ERIC', 'IVCBU', 'LILAK', 'NKGNW', 'OLPX', 'RXST', 'TRINZ', 'NN', 'ZJZZT', 'MNPR', 'LRFC', 'GSUN', 'TMC', 'SFIX', 'HURC', 'VSEC', 'CARA', 'CEG', 'CETUR', 'GNLN', 'ATRI', 'HSPOR', 'GOOGL', 'RNWWW', 'NBN', 'LYFT', 'MBLY', 'UBSI', 'FIAC', 'CUBA', 'SVRA', 'MLTX', 'MLECW', 'TURB', 'CCTSW', 'GYRE', 'AMST', 'AMZN', 'ITI', 'FOX', 'ILPT', 'PAVM', 'IIIV', 'SCOR', 'VRAR', 'WINVU', 'XOS', 'ALAR', 'VRDN', 'CRMT', 'SFNC', 'SONO', 'CNSL', 'ATIF', 'TVGNW', 'SNT', 'ARQ', 'ATNI', 'CHCO', 'ORRF', 'XTLB', 'ALDX', 'UROY', 'AVBP', 'CDIOW', 'RNAC', 'ASCBU', 'ESGR', 'GUTS', 'V', 'KRKR', 'SENEA', 'BLACR', 'SCHL', 'PHAR', 'LINC', 'PFTAU', 'SKGR', 'SKGRW', 'YYAI', 'VCSA', 'LKFN', 'CRKN', 'FDMT', 'GEG', 'UAL', 'WRNT', 'NYMTZ', 'MATH', 'NEWT', 'RTC', 'MESA', 'LYTS', 'DRIO', 'LULU', 'EDAP', 'REAL', 'CG', 'ASO', 'AVT', 'GAMC', 'IPXXW', 'CCCS', 'NMFCZ', 'BRKL', 'NTWK', 'PCRX', 'CRDO', 'INTEW', 'KC', 'JZ', 'SPWR', 'ZPTAW', 'TWOU', 'INSM', 'IMMX', 'NITO', 'CSWCZ', 'LILMW', 'ARDX', 'CDLX', 'MEDP', 'VSAC', 'CATY', 'BUJAW', 'EMLDW', 'LNKB', 'PCTTU', 'CCTS', 'MSGM', 'LANDM', 'NKSH', 'CNOB', 'STEC', 'TETEW', 'VSTEW', 'OFSSH', 'CETUW', 'BCML', 'DZSI', 'TMTCU', 'GAINL', 'VRRM', 'SDOT', 'HOOD', 'EML', 'BNAIW', 'NUVL', 'CNDT', 'FTAIM', 'BPYPM', 'GLACR', 'NEON', 'HRTX', 'CCLDP', 'IGTAR', 'CLRCR', 'PAGP', 'SBET', 'ACTG', 'ASUR', 'CVLT', 'WTMAR', 'XCUR', 'HD', 'AGAE', 'BOF', 'IPHA', 'ROAD', 'SLMBP', 'ONYX', 'SWAG', 'ZBAO', 'YYGH', 'HNVR', 'PARAA', 'RVSN', 'ATMVR', 'CPTN', 'MTTR', 'FRLAW', 'SJ', 'ONCT', 'CMRX', 'BTCY', 'PLYA', 'TECTP', 'USAU', 'NVEC', 'DTST', 'ETON', 'TFINP', 'LPRO', 'RCKTW', 'OCSL', 'TARA', 'DOW', 'HEAR', 'RETO', 'HEPA', 'CEAD', 'MCAAU', 'DSP', 'AREBW', 'GGAL', 'GOVX', 'ABLLL', 'FGF', 'RDFN', 'SMXT', 'APLMW', 'CLBTW', 'BLEUR', 'CMTL', 'TYRA', 'CRESW', 'LUCY', 'IMTE', 'ISPO', 'ORMP', 'FIACW', 'SHLS', 'XBIO', 'NWSA', 'BPMC', 'CCB', 'USGOW', 'FSBW', 'EXPE', 'KE', 'VALU', 'NSIT', 'QUIK', 'VCYT', 'MRNO', 'QSIAW', 'FITB', 'SRTS', 'EFOI', 'ACNT', 'STRL', 'GNFT', 'AMRN', 'TNGX', 'LGND', 'FONR', 'HUMAW', 'PCB', 'WSBC', 'AHCO', 'WAY', 'BLDP', 'CTCX', 'CRUS', 'MAT', 'TSBX', 'GAN', 'BLACU', 'OCUL', 'MNSBP', 'TRUP', 'ARM', 'MAQCU', 'CNXC', 'LCID', 'MIRA', 'CRML', 'CVGW', 'TIRX', 'HCTI', 'DERM', 'EWTX', 'EVO', 'IVVD', 'DAWN', 'COST', 'PAX', 'OPK', 'PNRG', 'DISTR', 'HRMY', 'SLRN', 'VFS', 'PEGY', 'GTHX', 'SILO', 'MDXH', 'SEATW', 'AGEN', 'GFS', 'ALSAW', 'ZIMV', 'IKT', 'CTHR', 'CENX', 'INBKZ', 'AAME', 'JVSAR', 'NRBO', 'ASRT', 'NPABW', 'NSSC', 'PPTA', 'HDSN', 'GLLIW', 'AIMBU', 'EVAX', 'AOGO', 'NB', 'CUE', 'GLLIU', 'PLXS', 'GNLX', 'ATEX', 'TFFP', 'OPCH', 'BANX', 'LAND', 'SGHT', 'GECCO', 'DHIL', 'NAUT', 'METCB', 'PWP', 'IBKR', 'BSVN', 'WHLRD', 'ISTR', 'TSBK', 'WMG', 'MARXU', 'BETR', 'CELUW', 'CLEU', 'DPRO', 'LSXMK', 'AZTA', 'CAUD', 'JBLU', 'NBTX', 'SDSTW', 'CINGW', 'GCT', 'LOCO', 'LFMD', 'RFACW', 'NFE', 'OBLG', 'LTRN', 'ABVX', 'ECDA', 'GT', 'CRNT', 'AFCG', 'TH', 'PI', 'PEGRW', 'ZNTL', 'JBHT', 'PLMI', 'ATXS', 'CASY', 'IPXX', 'OZKAP', 'HLXB', 'NAAS', 'NCNA', 'PRLHU', 'RYAAY', 'IDN', 'THCH', 'VBTX', 'VSTA', 'CREG', 'DRTSW', 'GRNQ', 'HIMX', 'INBX', 'RCM', 'CNTY', 'CREX', 'RKDA', 'VYNE', 'DYCQR', 'TLSIW', 'GRTS', 'GOGL', 'HROW', 'APLT', 'ASCB', 'SEVN', 'AACIW', 'SXTP', 'SVMH', 'DSGX', 'OCTO', 'AMKR', 'OB', 'AFRIW', 'BAYA', 'SNCRL', 'INAQW', 'ANSC', 'SHOT', 'RGF', 'CGON', 'IBCP', 'PCSA', 'FSLR', 'BZ', 'MAYS', 'COOT', 'COIN', 'IART', 'PIK', 'PWOD', 'TIVC', 'LOAN', 'ALOT', 'TDUP', 'CABA', 'EQIX', 'MOBX', 'AMBA', 'HOVR', 'NVA', 'NVCR', 'SGMA', 'TMTCR', 'NAMSW', 'FDBC', 'ABVC', 'GSIW', 'GALT', 'ENVX', 'ORGNW', 'EU', 'RVPHW', 'SSYS', 'ACLX', 'TALKW', 'TMCWW', 'TPGXL', 'BYFC', 'JFBRW', 'RPTX', 'NVEE', 'BYNOU', 'LTBR', 'SVA', 'MYPSW', 'RMCO', 'LQDA', 'ARCB', 'PESI', 'ETSY', 'FTFT', 'ILMN', 'SBFM', 'LI', 'RRBI', 'MYSZ', 'OPTN', 'ANSS', 'DIST', 'ADSEW', 'MIRM', 'LAB', 'GLPG', 'MOBXW', 'ITCI', 'RVPH', 'JAKK', 'PDCO', 'IRDM', 'HNNAZ', 'ATHA', 'AIRG', 'CODX', 'FUTU', 'PDLB', 'RILYL', 'TCRX', 'HUIZ', 'XMTR', 'SONDW', 'QTTB', 'DRVN', 'ADTX', 'ZION', 'ELTX', 'GPAK', 'ATSG', 'ENG', 'CADL', 'CSTL', 'LOPE', 'HCKT', 'TOI', 'VRNA', 'CTNT', 'NEPH', 'GEOS', 'SPPL', 'FIVN', 'MGX', 'SOGP', 'GRDIW', 'PRCT', 'WLDS', 'VIASP', 'QCOM', 'AROW', 'IROHW', 'NSTS', 'CTBI', 'AKBA', 'PSNY', 'CSTE', 'NTLA', 'PRTS', 'PIXY', 'FCBC', 'VBFC', 'KGEI', 'DFLIW', 'PINC', 'OCG', 'FEXDR', 'OSUR', 'AAGR', 'WRLD', 'IPA', 'RRGB', 'ALT', 'CLBT', 'CLRCU', 'GOGO', 'NTRP', 'ZYXI', 'SVIIR', 'GECC', 'INTS', 'MCRB', 'WDAY', 'MSBI', 'CUTR', 'EXPO', 'JUNE', 'KOD', 'LUXH', 'ESGL', 'YOTAU', 'BFRGW', 'FCUV', 'FTAIN', 'ATXI', 'BGNE', 'PXLW', 'GPATW', 'SEER', 'UVSP', 'SEZL', 'BLKB', 'GRAL', 'IBEX', 'KELYA', 'AVIR', 'MQ', 'SPKLW', 'ICUCW', 'GPRE', 'SWKH', 'TXN', 'DOX', 'TSAT', 'BLEUW', 'ARCC', 'CCSI', 'SLNO', 'BUJA', 'AMWD', 'BACK', 'HCP', 'TSEM', 'MCVT', 'APPS', 'PEP', 'HPKEW', 'FINW', 'WHFCL', 'PKOH', 'PRGS', 'DOCU', 'LSCC', 'MITAW', 'VIR', 'ADSK', 'EGHT', 'CTKB', 'RZLT', 'HKIT', 'GEHC', 'DUO', 'CSPI', 'SWIN', 'OPAL', 'BUSE', 'CELZ', 'BHAC', 'BTOG', 'CLLS', 'NEWTL', 'NRXP', 'OPTX', 'BANFP', 'FXNC', 'BMRA', 'MGIC', 'LGMK', 'WETH', 'IDYA', 'WENAW', 'FTII', 'PMVP', 'AUROW', 'AVPT', 'ELVN', 'TRMB', 'OFLX', 'PTLO', 'LAUR', 'MFICL', 'BHFAL', 'DSY', 'ZUMZ', 'VOXX', 'VMEO', 'RILYN', 'CLOER', 'AHI', 'ALTR', 'NVDA', 'MVIS', 'RSVRW', 'WLFC', 'WLDSW', 'DYAI', 'MAPS', 'TELA', 'EFSC', 'LAKE', 'TCBPW', 'IMTX', 'PMN', 'ADI', 'THAR', 'PDSB', 'GLMD', 'INDP', 'AERTW', 'PAYS', 'VSACW', 'BON', 'BCDA', 'GATEU', 'GDEN', 'GECCI', 'NRSNW', 'TPG', 'XRTX', 'OABIW', 'BRAG', 'CLFD', 'IRTC', 'SYPR', 'NVX', 'BBSI', 'COO', 'IZM', 'NRXPW', 'PWM', 'TRNS', 'MVST', 'CVIIU', 'STAF', 'SIDU', 'ANGI', 'WPRT', 'ROKU', 'GNTA', 'STRO', 'DKNG', 'HOWL', 'CORZW', 'MBOT', 'ARQQ', 'BRFH', 'IBRX', 'SEEL', 'PCTTW', 'TRVN', 'CVV', 'CBRG', 'CITE', 'PTPI', 'CTRN', 'PBM', 'TBLAW', 'SLDB', 'XLO', 'MDRRP', 'CCTG', 'IMAQ', 'ASML', 'FTIIW', 'LFWD', 'KDLYW', 'HPCO', 'GAQ', 'CROX', 'SOWG', 'IFBD', 'HELE', 'LRCX', 'LAZR', 'WATT', 'GOSS', 'FCFS', 'PROP', 'SPNS', 'ELEV', 'SABS', 'MICS', 'IAC', 'POCI', 'CCIXW', 'APPF', 'FCEL', 'GLSTW', 'MBUU', 'LSAK', 'PAHC', 'BATRK', 'BWAY', 'SLDPW', 'SVMHW', 'LMB', 'PRST', 'NTGR', 'CDNA', 'DYCQ', 'SPFI', 'ZS', 'FOXA', 'KITT', 'MDJH', 'MOVE', 'PLMJ', 'UTHR', 'ALGM', 'SVREW', 'CYN', 'LANC', 'ALBT', 'GLNG', 'NFLX', 'ARTL', 'CMPS', 'FROG', 'MDGL', 'VCTR', 'KITTW', 'FELE', 'LBTYK', 'LFUS', 'CLRC', 'ORKT', 'LVTX', 'RDNT', 'WABC', 'MKTX', 'XELAP', 'IRMD', 'FVCB', 'MSFT', 'ABTS', 'AIRT', 'STCN', 'VOXR', 'SURGW', 'COHU', 'CRBP', 'SPRY', 'NCNCW', 'NBSTU', 'RPD', 'RXT', 'MCHX', 'ADAP', 'TRS', 'ACAC', 'MSEX', 'AAOI', 'SMCI', 'GDC', 'WTMAU', 'KPTI', 'AFAR', 'MGNI', 'RCKY', 'SANA', 'VLCN', 'INTJ', 'ESPR', 'VGAS', 'CHSCL', 'FORLU', 'IVCAU', 'CNTM', 'RAIL', 'RGNX', 'PSTX', 'SCVL', 'SOUNW', 'FOSL', 'ISRL', 'COLM', 'NTNX', 'CGBDL', 'OPI', 'GWRS', 'HROWM', 'TBIO', 'PTC', 'ATPC', 'HTHT', 'GTACU', 'HOOK', 'MRKR', 'HTIA', 'NNDM', 'AUUD', 'JL', 'CSIQ', 'DRMAW', 'IMXI', 'NCRA', 'GRDI', 'NPABU', 'ONCY', 'BROG', 'MYGN', 'ORLY', 'EVGOW', 'THRY', 'BTSGU', 'OFIX', 'WYNN', 'FTNT', 'MVSTW', 'MOGO', 'NKGN', 'PFG', 'OSPN', 'PRFT', 'RNAZ', 'TWKS', 'AIRTP', 'TENKR', 'PGC', 'PGEN', 'OKYO', 'PEGA', 'ZOOZW', 'FATE', 'ADN', 'WBTN', 'OTEX', 'OSW', 'BIOX', 'MLCO', 'GLUE', 'VNET', 'GATEW', 'POET', 'CCLD', 'CCEP', 'DXPE', 'NOVVU', 'RANI', 'WEN', 'MODD', 'RCRT', 'GCTK', 'HCWB', 'FRLAU', 'GTIM', 'PTWO', 'VMCA', 'BFST', 'BWMN', 'SHFSW', 'ROCK', 'VSTM', 'SMTK', 'VABK', 'ICG', 'KARO', 'GLACU', 'BEEM', 'HTOO', 'OCEAW', 'KMDA', 'AVAH', 'UFPI', 'NXPI', 'AQST', 'SSBI', 'HHGC', 'VCNX', 'PCH', 'BLMN', 'VIVK', 'CVX', 'BRZE', 'MMAT', 'NVCT', 'POWW', 'RGC', 'NATH', 'TVGN', 'GSIT', 'ALCYU', 'AOSL', 'HEES', 'WNEB', 'IRAA', 'URBN', 'CVLG', 'EPRX', 'HIHO', 'LLYVK', 'LQR', 'ANGH', 'LATG', 'QUBT', 'PGHL', 'ISPR', 'XWEL', 'MYNA', 'IMTXW', 'NKLA', 'DCAP', 'EDIT', 'PTIXW', 'BFI', 'BLACW', 'CISO', 'DPCSU', 'FIVE', 'SHEN', 'CURI', 'DFLI', 'DNTH', 'ATLCP', 'GBBK', 'RMCF', 'FTEL', 'SONN', 'TANH', 'KSPI', 'PPYAW', 'TGAAW', 'VIOT', 'ARLP', 'JD', 'STBX', 'ARHS', 'EZGO', 'GABC', 'ALFUU', 'FEAM', 'GFAI', 'WLDN', 'OMAB', 'EMCGU', 'HPH', 'OABI', 'SCNI', 'FEIM', 'USGO', 'HAO', 'FANG', 'APYX', 'LEXXW', 'OCS', 'MOMO', 'CBFV', 'IMUX', 'HST', 'BAYAR', 'ACRS', 'IINNW', 'AEIS', 'AQMS', 'ATMCU', 'QSI', 'PHUN', 'TRINL', 'AILEW', 'SOHOB', 'KPLT', 'LNW', 'LOBO', 'RENEU', 'SPT', 'FLGT', 'JAMF', 'KSCP', 'STGW', 'ESTA', 'CCCC', 'SPRC', 'WHLRP', 'SNCY', 'NNAVW', 'ESGLW', 'TTMI', 'CFFN', 'INMB', 'FRPT', 'PBPB', 'TGAA', 'HITI', 'LOGC', 'AUBN', 'ARKO', 'EVRG', 'MSS', 'TRVG', 'OXLCZ', 'ASCBW', 'XTIA', 'CHKEW', 'ANSCU', 'VSEEW', 'PALI', 'SLE', 'AWH', 'JANX', 'LCFYW', 'PROV', 'SSNC', 'FATBB', 'MAR', 'WOOF', 'ZAPP', 'CIFRW', 'BCYC', 'OAKU', 'VNDA', 'RCAT', 'MTEN', 'NVOS', 'SHFS', 'ATMCW', 'LVRO', 'SANW', 'YMAB', 'LANDO', 'NMFC', 'EYEN', 'WALDW', 'ABVEW', 'EUDAW', 'QQQX', 'HQY', 'ELVA', 'SMBC', 'BVS', 'AMCX', 'ALZN', 'PBHC', 'ALGT', 'BPOPM', 'FRPH', 'SNY', 'NKTR', 'GLAC', 'LRHC', 'AMED', 'CRSR', 'ATGL', 'SCYX', 'SELX', 'UPC', 'GRFS', 'BPTH', 'PRPO', 'RCRTW', 'LKCO', 'ALCY', 'CGNX', 'FLIC', 'ABOS', 'RDUS', 'REKR', 'PFIS', 'MPWR', 'AZ', 'PGYWW', 'BKHA', 'AFBI', 'HUBG', 'BWIN', 'BIMI', 'ULBI', 'KRUS', 'ARKR', 'GOOG', 'RFAIU', 'ZIONP', 'RNA', 'AAGRW', 'RPRX', 'HMST', 'OCUP', 'ORGO', 'OTLK', 'QOMOU', 'AVDL', 'BLTE', 'DTSQU', 'KPRX', 'QQA', 'RFACU', 'ASTLW', 'STRR', 'MNYWW', 'GROMW', 'AEZS', 'SISI', 'ARKOW', 'NXPL', 'COGT', 'INGN', 'MDLZ', 'GLSTR', 'NIVF', 'QETAR', 'ONDS', 'SLRX', 'CDMO', 'CHRD', 'FLYE', 'CDT', 'JVSAU', 'EFSCP', 'QETA', 'ARTW', 'CDXS', 'NWPX', 'DTSS', 'MYRG', 'ORIC', 'ALTI', 'GEN', 'BNR', 'HFFG', 'CGC', 'VMAR', 'MRK', 'NRC', 'TLPH', 'EVLVW', 'UHG', 'KO', 'BMRC', 'ITOS', 'EXPI', 'CYCC', 'ROCLW', 'XEL', 'PERI', 'OXSQG', 'SNSE', 'DRS', 'OSS', 'GSMGW', 'VINC', 'CFB', 'SMPL', 'ELTK', 'GTAC', 'DPCSW', 'DHAIW', 'BLZE', 'AUPH', 'BIVI', 'CMLS', 'DAVE', 'FCNCO', 'BREA', 'FER', 'GAMB', 'KXIN', 'ASLE', 'ATRC', 'LSXMB', 'MEGL', 'OLMA', 'PFTA', 'PLMIU', 'CPSS', 'VIRT', 'MGRX', 'ATNFW', 'PAVMZ', 'RDCM', 'XFINU', 'DUOL', 'KLTR', 'ELUT', 'UTMD', 'MSSAW', 'LNSR', 'AMAL', 'RILYZ', 'TFSL', 'BGXX', 'KRNT', 'TALK', 'CHSN', 'JOUT', 'KQQQ', 'KVAC', 'MAPSW', 'XRX', 'CLWT', 'SKGRU', 'CKPT', 'OXLCL', 'OMGA', 'BTCS', 'BLUE', 'PLAY', 'SLGL', 'TNYA', 'IBACR', 'INVZ', 'MATW', 'FULTP', 'NOVVW', 'RGEN', 'AMGN', 'SKYX', 'PKBK', 'CSGS', 'ABAT', 'WASH', 'LBTYA', 'GILD', 'ACLS', 'IVA', 'STAA', 'XFOR', 'ADSE', 'NMRK', 'SPWH', 'IVCAW', 'BEAM', 'FFIE', 'WHLR', 'SGRP', 'PEBO', 'PETWW', 'ATEC', 'GYRO', 'MCAA', 'MDAI', 'EBTC', 'CDTX', 'HSCSW', 'CTXR', 'EPSN', 'VERI', 'SAITW', 'AUR', 'BIDU', 'FLXS', 'SAIA', 'UDMY', 'KROS', 'NA', 'SASR', 'EBAY', 'ZURAW', 'XTKG', 'AFYA', 'CMAXW', 'VVPR', 'INTE', 'TTWO', 'TEAM', 'SPKL', 'OMH', 'BLNK', 'INBS', 'MFI', 'ZWZZT', 'TERN', 'NMIH', 'GRI', 'KINS', 'NBBK', 'NSPR', 'TCBIO', 'WESTW', 'INCY', 'ALCE', 'HYZN', 'ANGO', 'CHKP', 'RGTI', 'CNTA', 'FRME', 'HNNA', 'VLY', 'BAND', 'EMLDU', 'FEXD', 'MGPI', 'TSLA', 'WINA', 'PYXS', 'CWD', 'NXPLW', 'SGA', 'CPSH', 'EDUC', 'NSYS', 'DOYU', 'AFARW', 'RLAY', 'WIMI', 'DVAX', 'MAQCW', 'NCI', 'APRE', 'ADVWW', 'FRAF', 'JMSB', 'CDW', 'ACRV', 'CFFSW', 'PWUPW', 'VERU', 'ZENV', 'LIND', 'AMPG', 'CRVL', 'MAQC', 'NAII', 'AEMD', 'XBPEW', 'AUUDW', 'MASI', 'ULY', 'CDTG', 'VALN', 'OXSQZ', 'GECCZ', 'LE', 'XFINW', 'PRAX', 'GRRR', 'IMMP', 'CLMT', 'GAMCU', 'IDXX', 'MBINM', 'SBLK', 'IEP', 'LUCYW', 'EAST', 'MCBS', 'RGTIW', 'METCL', 'MOBBW', 'SMLR', 'ICCH', 'AIXI', 'CSBR', 'FLYW', 'LTRX', 'HYFM', 'RGLS', 'LRMR', 'STI', 'UPBD', 'ASPI', 'MCBC', 'FYBR', 'TWG', 'KVHI', 'WGSWW', 'IMRX', 'QXO', 'PTSI', 'PPSI', 'CFFSU', 'SLAB', 'LIDR', 'CAPR', 'BHRB', 'RFAI', 'CRGX', 'BRLSW', 'ALGS', 'APM', 'AGFY', 'BJRI', 'XNET', 'SAGE', 'MOND', 'KHC', 'LSBPW', 'CFBK', 'MNTS', 'CHRS', 'DCGO', 'OSBC', 'PRLHW', 'SABSW', 'ADTN', 'TATT', 'HOFT', 'IBTX', 'BBLGW', 'HOLO', 'BBGI', 'CZR', 'CYTO', 'WAVE', 'NCTY', 'NDAQ', 'ALGN', 'CCRN', 'ACHV', 'DDI', 'BPYPO', 'AYRO', 'CRDL', 'FSFG', 'TLF', 'UTSI', 'CORZZ', 'PEGRU', 'ROOT', 'BAER', 'NRDS', 'WWD', 'IRBT', 'CWST', 'OTLY', 'CREVW', 'SHOO', 'ARBB', 'MBWM', 'DRUG', 'LNTH', 'PEGR', 'CMMB', 'DASH', 'UEIC', 'RVMDW', 'BLBX', 'ROCL', 'RMBI', 'IMCC', 'BGFV', 'ADAG', 'FFIC', 'CERE', 'PFC', 'EXC', 'CBNK', 'PBFS', 'CGEM', 'SGC', 'SLRC', 'AMSC', 'PECO', 'CLOV', 'RAND', 'LIXTW', 'MCRI', 'CRGOW', 'AXDX', 'TRML', 'HMNF', 'TLRY', 'GHSI', 'TWFG', 'CNOBP', 'ICON', 'SBUX', 'SY', 'CBRL', 'BTCT', 'HHGCR', 'HSII', 'PLAO', 'SPCB', 'NUWE', 'TETEU', 'PSTV', 'VTYX', 'DGLY', 'HNRG', 'IMMR', 'VCEL', 'AACIU', 'AOGOU', 'FAAS', 'CDRO', 'CPHC', 'BSFC', 'JSPRW', 'DDOG', 'RVYL', 'DRMA', 'CCAP', 'WTO', 'COLB', 'RLMD', 'ICHR', 'SYBX', 'TRIP', 'ESSA', 'UPWK', 'FMSTW', 'VKTX', 'DSYWW', 'PGNY', 'LOGI', 'CETX', 'MCAG', 'ARTNA', 'ALSAU', 'KTTA', 'AQUNR', 'FAST', 'SBSI', 'EUDA', 'AEYE', 'GERN', 'SWIM', 'RECT', 'NCNO', 'MARXR', 'FAMI', 'TBMCR', 'BTCTW', 'BZFDW', 'QMCO', 'IINN', 'RAPP', 'GLSTU', 'VMCAW', 'OXSQ', 'OMIC', 'RMTI', 'VERA', 'WKSPW', 'DCBO', 'SRM', 'BVFL', 'MKSI', 'LVLU', 'BAERW', 'SANG', 'SOPA', 'CLNE', 'NWBI', 'SHBI', 'WHLM', 'JYNT', 'BTBT', 'MDXG', 'HAS', 'HAYN', 'HBANM', 'ISRLU', 'EOSE', 'LPCN', 'NEXT', 'FTCI', 'HBANP', 'RDZN', 'SNCR', 'NIVFW', 'GENK', 'LANDP', 'TRNR', 'ACET', 'CAPT', 'HLP', 'VECO', 'BOXL', 'SBT', 'YIBO', 'LBRDA', 'LEGH', 'NXL', 'SOPH', 'GEGGL', 'SGMO', 'VLGEA', 'WNW', 'CHK', 'FRES', 'SVRE', 'CAR', 'NVAWW', 'PMEC', 'WB', 'WKME', 'AGYS', 'YHGJ', 'DLO', 'FLNT', 'HWKN', 'WBUY', 'TNXP', 'NECB', 'GRPN', 'AIRJ', 'ETHA', 'MARA', 'WBD', 'ESEA', 'LEDS', 'INDI', 'SLS', 'AIMD', 'EMKR', 'ROMA', 'LUCD', 'ONVO', 'RELL', 'PCSC', 'CTNM', 'SFM', 'AFARU', 'TPIC', 'BTMWW', 'VC', 'HTLFP', 'OAKUU', 'BLAC', 'CRESY', 'CSWI', 'RAPT', 'TRST', 'APEI', 'NDSN', 'BMBL', 'NUVOW', 'HRZN', 'AVGR', 'BTSG', 'IMAQW', 'SKWD', 'PBMWW', 'THRM', 'TMDX', 'BLRX', 'MOLN', 'AVNW', 'SCKT', 'WVE', 'WKEY', 'CRMLW', 'SRPT', 'CYBR', 'TRMK', 'DXR', 'VVOS', 'BRKH', 'ENSC', 'PRLH', 'LITM', 'MBRX', 'QLGN', 'TRMD', 'DGICB', 'LEGN', 'CIGI', 'UCBIO', 'FSTR', 'CVAC', 'YQ', 'ZKIN', 'ARCT', 'BRLS', 'VIGL', 'AFRI', 'SLN', 'SOBR', 'AOUT', 'CCNE', 'APLM', 'AUGX', 'HAIA', 'TPCS', 'SYRA', 'LMFA', 'LVROW', 'MTEX', 'YOTAR', 'FTAI', 'OVLY', 'AQB', 'SCSC', 'HOLOW', 'FSEA', 'FFIEW', 'GLSI', 'LQDT', 'ABL', 'WEYS', 'AEI', 'FSBC', 'VGASW', 'MERC', 'PXSAW', 'HCSG', 'OZK', 'TBNK', 'HROWL', 'RCEL', 'SLM', 'MFIC', 'CRNX', 'LFVN', 'TAOP', 'SCWO', 'TLGYW', 'TRDA', 'WAVSU', 'COKE', 'HRYU', 'WERN', 'VYGR', 'TLN', 'PRTH', 'PLBY', 'ATMC', 'IBAC', 'DLTH', 'NTCT', 'MPB', 'NYMTI', 'BCAB', 'MAXN', 'RBB', 'SEED', 'INVZW', 'SLNHP', 'EVGR', 'FLNC', 'YOTA', 'FTAIO', 'FWONA', 'NCPL', 'UG', 'CYRX', 'KZIA', 'BLMZ', 'TLSA', 'IKNA', 'BMEA', 'ACGLN', 'LCUT', 'INZY', 'NCSM', 'FRSX', 'RPAY', 'HUDAR', 'OCGN', 'META', 'ULH', 'MARPS', 'CME', 'QLYS', 'SWAGW', 'WVVIP', 'WSFS', 'FNVTU', 'BOWNR', 'CASI', 'DXYN', 'NVVEW', 'PETS', 'PSMT', 'HYPR', 'OXLCI', 'ONB', 'OP', 'CAMT', 'WW', 'BYND', 'VSAT', 'SDA', 'RBCAA', 'OKTA', 'CRWD', 'AITRU', 'BHFAM', 'SEAT', 'AAL', 'BRKR', 'CCTSU', 'PPBT', 'DSGR', 'AXP', 'PROCW', 'HGBL', 'CDAQ', 'ESQ', 'HUGE', 'FEXDU', 'HWBK', 'NATR', 'MGEE', 'NTES', 'NDRA', 'PRAA', 'JCSE', 'SWTX', 'FKWL', 'PNFPP', 'SNTG', 'IPAR', 'UCL', 'CGTX', 'EOSEW', 'POWL', 'AXSM', 'EDTK', 'VEEE', 'MBNKP', 'BETRW', 'FUNC', 'PCTY', 'GPAT', 'ALSAR', 'CSCO', 'BZFD', 'BCGWW', 'KRRO', 'BLIN', 'MARX', 'TORO', 'ONYXW', 'ARQT', 'DHCNL', 'POWWP', 'DLTR', 'ELWS', 'JJSF', 'LIDRW', 'APDN', 'BCOW', 'NKE', 'QOMO', 'RVMD', 'ESOA', 'TILE', 'TFIN', 'THFF', 'SNEX', 'HSAI', 'KALV', 'AMRX', 'LGHL', 'HAIN', 'NVFY', 'GAIA', 'LINE', 'ROCLU', 'SPECW', 'STRM', 'BFIIW', 'BMR', 'KFFB', 'VSTE', 'YTRA', 'AITRR', 'APLS', 'AFJK', 'MTRX', 'KRYS', 'KTCC', 'BAOS', 'AFRM', 'MNDO', 'SOHO', 'NXGL', 'UFPT', 'CHSCO', 'CCOI', 'INMD', 'CYTH', 'SRZNW', 'CCNEP', 'DUOT', 'FREE', 'CNTB', 'OCCIO', 'FARO', 'PXSAP', 'SXTC', 'PAYX', 'XGN', 'GO', 'ACACW', 'PAVS', 'FIZZ', 'NMRA', 'SNDL', 'CXAI', 'SNOA', 'ALLK', 'DECAU', 'DLHC', 'VSSYW', 'ACGLO', 'SHIM', 'BANF', 'KYMR', 'CDXC', 'IVCPW', 'LOVE', 'SMTI', 'EGAN', 'TMCI', 'FOSLL', 'PFXNZ', 'WTFC', 'MCD', 'UNH', 'CMCSA', 'RRR', 'TRINI', 'ALRS', 'PRVA', 'MMVWW', 'PRSTW', 'RIVN', 'TECH', 'LZ', 'DECAW', 'HBIO', 'FSV', 'COSM', 'DXLG', 'MDAIW', 'MRVL', 'MNKD', 'NYMT', 'RFAIR', 'AMPGW', 'ABUS', 'PTCT', 'ENTX', 'ULCC', 'DBGI', 'OCC', 'ASTSW', 'ICCT', 'FBYDW', 'STSSW', 'WKSP', 'ACON', 'CARG', 'TKLF', 'PCVX', 'GANX', 'TIGR', 'NNOX', 'SVC', 'CPLP', 'DCOMP', 'HSON', 'OPRX', 'USIO', 'ARTV', 'AY', 'BCSA', 'PPYAU', 'VLYPP', 'CARV', 'RELIW', 'EEIQ', 'EDRY', 'INBK', 'IROQ', 'ACHC', 'HYZNW', 'MRM', 'WTMA', 'CHI', 'STSS', 'NWTNW', 'NTRB', 'LPAAU', 'FORM', 'IMVT', 'AITR', 'BCRX', 'SHIP', 'TRUG', 'PSHG', 'MEIP', 'NYMTM', 'GSHD', 'BLDEW', 'ONCO', 'DARE', 'CDAQW', 'NICK', 'SYBT', 'ZOOZ', 'BCAL', 'DISTW', 'JFBR', 'MCAGR', 'SMFL', 'TETE', 'PG', 'MNTX', 'SNDX', 'ATOS', 'LSTR', 'NYMTN', 'ALSA', 'LION', 'GLYC', 'WDFC', 'TW', 'LINK', 'SPGC', 'BRLT', 'CHNR', 'CTLP', 'AIP', 'BHFAO', 'FBLG', 'SSP', 'UBFO', 'CRTO', 'LCFY', 'AVO', 'HUDI', 'HON', 'GGR', 'TCBS', 'PRE', 'AVAV', 'IXHL', 'FSUN', 'APXIU', 'IVCB', 'METC', 'SNAXW', 'MGOL', 'IGTAW', 'LICN', 'CEADW', 'FIACU', 'MITAU', 'PBYI', 'POOL', 'STEP', 'TNONW', 'PODD', 'SAMG', 'XBP', 'TSHA', 'HEPS', 'HUDA', 'PLRX', 'SMSI', 'VLYPO', 'ADV', 'ICCM', 'LEE', 'UPXI', 'CRGO', 'MDIA', 'AMTX', 'HSDT', 'VREX', 'MNDY', 'NTBL', 'BOLT', 'CRBU', 'IMRN', 'INHD', 'MIST', 'VFF', 'ICAD', 'DUETU', 'RWAY', 'BTBD', 'RARE', 'RVSNW', 'HOUR', 'FA', 'ALVO', 'MACAW', 'BPRN', 'GIPRW', 'CLDX', 'CLPS', 'LUNRW', 'MORF', 'MRIN', 'OCCIN', 'SNPS', 'UNCY', 'GECCM', 'CBUS', 'MULN', 'XNCR', 'AEAEW', 'MRNOW', 'SDAWW', 'NXU', 'TLS', 'VWEWW', 'PBBK', 'UCTT', 'WTW', 'JACK', 'CINF', 'GROM', 'HUBCZ', 'TRSG', 'AEHR', 'ROIV', 'KACLU', 'CGBD', 'PAA', 'DAVEW', 'PZZA', 'IRAAW', 'DBGIW', 'IOBT', 'ASPS', 'GDSTR', 'RSLS', 'RNXT', 'NWTN', 'VIRC', 'NUZE', 'HBANL', 'MDRR', 'IMPP', 'YOTAW', 'ZLS', 'RMNI', 'CVII', 'CVBF', 'HBAN', 'NNBR', 'FNKO', 'MMM', 'IPXXU', 'OPT', 'GLLI', 'PYCR', 'MINDP', 'EVLV', 'ITRM', 'RSSS', 'KRNLW', 'TCBI', 'DUETW', 'CHMG', 'API', 'MACI', 'OPTXW', 'NAOV', 'TKNO', 'EVGO', 'INVA', 'PATK', 'TUSK', 'AENTW', 'PFBC', 'PPIH', 'LSB', 'FENC', 'SEIC', 'CSQ', 'MLGO', 'AACI', 'CURIW', 'IGTA', 'LYT', 'MKTW', 'OPEN', 'PNFP', 'TGL', 'KTRA', 'CNSP', 'DOMH', 'TOUR', 'TYGO', 'LIQT', 'AEP', 'CSGP', 'LFCR', 'MSTR', 'ATAI', 'URGN', 'CRON', 'VRM', 'WFCF', 'ASCBR', 'NKTX', 'LSH', 'APVO', 'ZVRA', 'CMND', 'MMLP', 'SLNH', 'GATE', 'STKS', 'ADUS', 'HTLD', 'BFRI', 'APTO', 'CNFR', 'TOMZ', 'LESL', 'KNSA', 'LXEO', 'RGS', 'CRM', 'CMAX', 'AGIX', 'LFLYW', 'AYTU', 'BKNG', 'EVGRW', 'CMCT', 'FITBO', 'SENEB', 'KTTAW', 'DIS', 'NETDW', 'SANM', 'THCP', 'CRVS', 'OLED', 'HUBC', 'FTIIU', 'SGML', 'MTSI', 'GIFI', 'BLEU', 'JRVR', 'ROIC', 'SPRO', 'LIFWW', 'DXCM', 'SIEB', 'ZEUS', 'ZJYL', 'HFWA', 'SITM', 'GLBS', 'HSPOU', 'IFRX', 'ICLK', 'TCPC', 'RMBS', 'DNUT', 'MNTSW', 'SNAL', 'CYTK', 'AZPN', 'QNRX', 'SGD', 'GLAD', 'VFSWW', 'LSBK', 'CETU', 'CITEW', 'GOCO', 'TELO', 'BAYAU', 'CLGN', 'TER', 'NBSTW', 'BL', 'BKHAU', 'MASS', 'CMPOW', 'GTLB', 'SAFT', 'GDSTW', 'BILI', 'NVEI', 'HHGCU', 'JNJ', 'PCAR', 'GLBE', 'CNET', 'IONR', 'ENPH', 'MOFG', 'JBSS', 'PPBI', 'ESGRP', 'STOK', 'SURG', 'TBLA', 'SAI', 'NDLS', 'CIFR', 'CJET', 'HTBI', 'ICFI', 'HLNE', 'NWS', 'FRBA', 'TCBP', 'FGBIP', 'QTI', 'DHCNI', 'RENB', 'VITL', 'GOODO', 'GLADZ', 'SFST', 'WEST', 'HCVIU', 'CORZ', 'JG', 'TECX', 'TTSH', 'CLAR', 'ZXZZT', 'FHB', 'GHIX', 'ALNT', 'BCOV', 'EMCG', 'THTX', 'TOP', 'CALM', 'IONS', 'CVCO', 'CHX', 'CWBC', 'IPSC', 'NEWTI', 'OXLC', 'COLL', 'LUNG', 'IRWD', 'SVIIU', 'CPTNW', 'ACABU', 'IAS', 'TREE', 'GMAB', 'KACLR', 'MRNA', 'ACIW', 'CHSCP', 'CZWI', 'ETNB', 'CECO', 'DADA', 'GIPR', 'INCR', 'MSSAR', 'FULT', 'HYMCL', 'NERV', 'MSAIW', 'ASTH', 'NWGL', 'ABNB', 'QRTEA', 'SCPH', 'TOIIW', 'CACC', 'FTAIP', 'MTCH', 'TASK', 'ALLO', 'CAPTW', 'FNLC', 'XP', 'ONYXU', 'RENE', 'CNFRZ', 'CRSP', 'CJJD', 'PTWOU', 'EXEL', 'RXRX', 'NEXN', 'RFAC', 'HBCP', 'NMHI', 'ABEO', 'AISP', 'MNY', 'PTEN', 'EXAI', 'SBAC', 'AUID', 'NARI', 'ERNA', 'CCIXU', 'PPC', 'EMCGW', 'TCTM', 'CRAI', 'STER', 'ECBK', 'KACLW', 'SIFY', 'EPOW', 'BUJAR', 'AMIX', 'JDZG', 'RWAYZ', 'TARS', 'WINT', 'NEOG', 'ATMVU', 'CXDO', 'PGY', 'SINT', 'ITRN', 'SSKN', 'EBMT', 'HAFC', 'OSIS', 'WAFD', 'PTWOW', 'SHYF', 'GBIO', 'NVTS', 'ZG', 'GWAV', 'MNST', 'MNSB', 'JZXN', 'RDI', 'EPIX', 'AFJKU', 'SPOK', 'SYM', 'ENGN', 'RWAYL', 'SNBR', 'ESLT', 'KLAC', 'LNZA', 'TNFA', 'FRMEP', 'CRCT', 'PSNYW', 'ASYS', 'ARRY', 'NAMS', 'PRPH', 'LX', 'CEROW', 'SNES', 'YI', 'PHAT', 'WILC', 'OTTR', 'RKLB', 'TENKU', 'ERAS', 'DRCT', 'VSACU', 'BCPC', 'CFFI', 'AGNCP', 'BTMD', 'ZD', 'FIBK', 'MTEM', 'OPOF', 'NTRBW', 'INAQU', 'USLM', 'TTOO', 'ACT', 'CNTG', 'CHW', 'NMHIW', 'ESLA', 'AREC', 'GGROW', 'FCCO', 'RLYB', 'FSHPU', 'HSTM', 'SHPH', 'TTEK', 'USEG', 'ODVWZ', 'RUN', 'MSSA', 'AOGOW', 'CING', 'CLOE', 'LGCL', 'TBLD', 'Z', 'ESGRO', 'BOCNW', 'NVAC', 'WAFDP', 'WMPN', 'KIRK', 'VINO', 'EDBL', 'GOODN', 'UBX', 'TVTX', 'ROP', 'ZTEK', 'CTSO', 'CHSCN', 'EXAS', 'JCTCF', 'MIND', 'WISA', 'STX', 'BCSAW', 'DORM', 'LBRDP', 'OPRA', 'TLIS', 'VOR', 'VRAX', 'AEHL', 'GAINZ', 'BJDX', 'DOGZ', 'SLDP', 'ARGX', 'SMID', 'STLD', 'WGS', 'ECXWW', 'CCLDO', 'HOFVW', 'AGBAW', 'BSBK', 'EMCGR', 'MAMO', 'ACST', 'CPRT', 'HSCS', 'OLLI', 'FIP', 'CYTHW', 'PRTC', 'VRMEW', 'AMAT', 'ZM', 'CNTX', 'MRUS', 'GDRX', 'COCH', 'FLWS', 'IMKTA', 'MTEK', 'QSG', 'ASND', 'TXG', 'MGRC', 'LAMR', 'BWBBP', 'FORLW', 'PXS', 'MTC', 'ATMCR', 'SSRM', 'EXTR', 'VBIV', 'LGCB', 'FMST', 'ERIE', 'FUSB', 'KLIC', 'BOWN', 'KA', 'QOMOR', 'ICCC', 'AXTI', 'TWST', 'CZNC', 'MFIN', 'BRKHW', 'ABSI', 'BLEUU', 'BECN', 'ANTE', 'WRAP', 'SXTPW', 'NWFL', 'STRT', 'GRAB', 'ANDE', 'MIDD', 'BOKF', 'HPK', 'NPCE', 'AADI', 'LTRY', 'FFIV', 'SLNG', 'JTAI', 'STTK', 'CLOEU', 'CWCO', 'CDTTW', 'AVTE', 'HOPE', 'CELH', 'WSBF', 'QRVO', 'AQU', 'DMAC', 'TOYO', 'WSBCP', 'SONM', 'GENE', 'LMAT', 'PEPG', 'GIII', 'HQI', 'PASG', 'TBBK', 'RMCOW', 'CEVA', 'LILA', 'TXRH', 'MBINO', 'PULM', 'THCPU', 'ZVSA', 'BTAI', 'QRTEP', 'PLSE', 'ECX', 'APXIW', 'BBCP', 'EMBC', 'CPRX', 'HURN', 'DGICA', 'EKSO', 'LGVN', 'NNAGW', 'ARWR', 'SCLXW', 'IREN', 'LSXMA', 'SBGI', 'FNVT', 'PSNL', 'FWRD', 'IVAC', 'BSET', 'CVGI', 'SILC', 'ASTC', 'HCVIW', 'CDAQU', 'TXMD', 'HBT', 'RMBL', 'SHCRW', 'HUT', 'RUMBW', 'FOLD', 'CLIR', 'KZR', 'DCOM', 'CGO', 'LMNR', 'PYPL', 'QCRH', 'GROW', 'GCBC', 'LUNR', 'VXRT', 'TLGYU', 'RCON', 'TRAW', 'FRGT', 'ATLX', 'PWUP', 'FNWD', 'CSLRW', 'COCHW', 'NVAX', 'INAB', 'PTON', 'KWE', 'IGMS', 'JRSH', 'MPAA', 'GDEV', 'BPYPN', 'SRDX', 'PSIG', 'NYXH', 'REGCP', 'THRD', 'SOUN', 'INTU', 'SDST', 'WALD', 'FTDR', 'GDST', 'SMTC', 'FREEW', 'IESC', 'ORGN', 'KOPN', 'BCBP', 'QTRX', 'SUNS', 'TIL', 'FISI', 'PDD', 'AUDC', 'ONFOW', 'RUM', 'PLMR', 'ALTS', 'ISPOW', 'REG', 'RDVT', 'PXDT', 'SATLW', 'BOLD', 'SHCR', 'KDLY', 'ROST', 'EWCZ', 'BHACU', 'TMTC', 'WHF', 'OXBRW', 'GAIN', 'BATRA', 'MRAM', 'MLYS', 'ACXP', 'CHKEL', 'TROW', 'SRCE', 'NTRS', 'GRIN', 'CLNNW', 'BNGO', 'MGNX', 'CETY', 'NWE', 'RSVR', 'ERII', 'CREV', 'CVKD', 'BRACU', 'ANTX', 'TRVI', 'PAYO', 'BENFW', 'SIRI', 'FAASW', 'AMLX', 'MITK', 'ZYME', 'BKR', 'LRE', 'ZLSWW', 'UONE', 'INAQ', 'LGHLW', 'PLL', 'PLMIW', 'SIMO', 'TCOM', 'RUSHA', 'LGO', 'PLAB', 'BCAN', 'ENSG', 'BTM', 'GOVXW', 'IPW', 'LFLY', 'EGIO', 'AENT', 'GPATU', 'CSX', 'INTZ', 'RENT', 'REVB', 'UXIN', 'TGTX', 'TRIN', 'BCLI', 'QNST', 'YORW', 'MHUA', 'NODK', 'BHFAN', 'BIAFW', 'TITN', 'NETDU', 'OXLCP', 'VANI', 'CRIS', 'GDS', 'MRX', 'NNAGU', 'REVBW', 'SIGI', 'AGIO', 'FBRX', 'ATXG', 'NOVV', 'PRENW', 'RENEW', 'NUVO', 'CGABL', 'AXNX', 'ACAD', 'ALKS', 'AVDX', 'CHEK', 'VEON', 'NUKKW', 'IROHU', 'QDEL', 'JWEL', 'HHGCW', 'ALVOW', 'CBAN', 'TTGT', 'GH', 'AEAEU', 'WBA', 'MCFT', 'PETQ', 'CHY', 'QMMM', 'TWIN', 'RFIL', 'BIAF', 'TCMD', 'UGRO', 'CPZ', 'IHRT', 'GMM', 'CTMX', 'ADPT', 'BMRN', 'EGRX', 'ISRLW', 'NICE', 'EDBLW', 'FGFPP', 'SPTN', 'LDTC', 'TENX', 'EMLD', 'GODNR', 'GASS', 'GODNU', 'RMR', 'NBTB', 'SWVLW', 'COCP', 'MRSN', 'IVP', 'GLBZ', 'JVSA', 'UCAR', 'NEWTG', 'SQFT', 'PRCH', 'BFRG', 'STKH', 'BRTX', 'FDUS', 'CUBWU', 'ACACU', 'ALRN', 'FCNCA', 'KLXE', 'INDB', 'LASR', 'PRTG', 'INVO', 'STIM', 'MFH', 'NIU', 'TAIT', 'XOMAP', 'AMLI', 'BZUN', 'CAT', 'NIOBW', 'FBIOP', 'OTRK', 'CGBSW', 'ASTS', 'IRAAU', 'DUET', 'NVNIW', 'TBRG', 'VERO', 'XYLO', 'MDB', 'PLUS', 'XXII', 'QURE', 'IXAQW', 'ONMD', 'ADXN', 'CENT', 'BIIB', 'PSEC', 'ULTA', 'INTA', 'GLST', 'KVACW', 'BYNOW', 'MACA', 'GLPI', 'NXGLW', 'SNPX', 'TCBK', 'VERX', 'LWAY', 'JPM', 'USCB', 'SERA', 'UBXG', 'STNE', 'ANIP', 'HYMC', 'SGBX', 'LUMO', 'YOSH', 'FUFU', 'ZIONL', 'RR', 'TTEC', 'PRDO', 'JNVR', 'STRRP', 'ACIC', 'DIBS', 'NEO', 'SYTA', 'PAL', 'VRNS', 'AIMAU', 'VRSN', 'GHRS', 'GEVO', 'GNSS', 'VTNR', 'ONBPO', 'REAX', 'HONE', 'AIRS', 'CMPO', 'MESO', 'PLAOU', 'PMD', 'MXL', 'SILK', 'PROC', 'HWCPZ', 'ADIL', 'REBN', 'AILE', 'APPN', 'SSIC', 'LPTX', 'FFNW', 'ECOR', 'ADEA', 'AACG', 'BPYPP', 'LIXT', 'KVACU', 'VRCA', 'GRRRW', 'FITBI', 'IMPPP', 'IVDA', 'FTRE', 'FTHM', 'TENB', 'HOVNP', 'IMNM', 'TEM', 'SATL', 'VNOM', 'ZEO', 'AEAE', 'REFR', 'ANY', 'GAINN', 'CALC', 'BNIXR', 'DYCQU', 'LIN', 'BCTXW', 'HUMA', 'FPAY', 'CALT', 'VRA', 'JSPR', 'BHACW', 'GTBP', 'OVBC', 'FHTX', 'ACAB', 'BKHAR', 'GTX', 'SRCL', 'VRSK', 'ZI', 'QRTEB', 'PACB', 'CCIX', 'KELYB', 'CRVO', 'RPID', 'YNDX', 'GOOD', 'EURKU', 'ECPG', 'SHMDW', 'ELBM', 'FULC', 'LIFWZ', 'GRVY', 'BLFS', 'RGLD', 'LFST', 'HNST', 'BRNS', 'PDYN', 'FWRG', 'MCAGU', 'FORL', 'CERT', 'CNXN', 'CRMD', 'TC', 'MGTX', 'GDYN', 'TRON', 'VSME', 'DCTH', 'FGEN', 'MRVI', 'SNAX', 'CLRCW', 'AMRK', 'BASE', 'MEOH', 'ENLT', 'GSM', 'DALN', 'EVTV', 'OAKUR', 'VOD', 'FCNCP', 'DBVT', 'ENLV', 'SWKHL', 'TRIB', 'NISN', 'CLSD', 'CLSK', 'CONN', 'REFI', 'APWC', 'NVNO', 'AGNC', 'NOVT', 'OMEX', 'QLI', 'KACL', 'GDTC', 'EH', 'VIAV', 'ENVB', 'PTIX', 'SGMT', 'FTLF', 'INSE', 'PTMN', 'GSBC', 'VS', 'AQUNU', 'LITE', 'HTZWW', 'ARVN', 'QOMOW', 'COYA', 'LIF', 'MSSAU', 'OPXS', 'AGMH', 'DTI', 'CMPR', 'SERV', 'BIOR', 'CENN', 'FNGR', 'RILYK', 'HSPOW', 'NGNE', 'HSIC', 'MRCY', 'XFIN', 'CDZI', 'ECDAW', 'SUUN', 'CNEY', 'AVPTW', 'PUBM', 'ALRM', 'FANH', 'ENTA', 'FATBP', 'BPOP', 'SGRY', 'SATS', 'BWEN', 'ICUI', 'RICK', 'ITIC', 'APA', 'VTRS', 'ESLAW', 'CHRW', 'ALEC', 'JXJT', 'OCFC', 'AVXL', 'BOSC', 'EEFT', 'HHS', 'HLMN', 'HTZ', 'XERS', 'TGAAU', 'ATHE', 'IRON', 'CLRB', 'PARA', 'CORT', 'INKT', 'GTACW', 'CHR', 'REE', 'ASST', 'FITBP', 'IBM', 'LASE', 'FUFUW', 'TSVT', 'SNGX', 'SLAMU', 'XOMA', 'CBSH', 'NTRA', 'GBBKR', 'IRIX', 'MODV', 'ZCAR', 'SIGIP', 'JAZZ', 'PRQR', 'FWONK', 'DTIL', 'VRPX', 'BBLG', 'BCG', 'AZN', 'CELC', 'LGIH', 'SPRB', 'TAYD', 'CSWC', 'SOND', 'MDBH', 'HOFV', 'SPEC', 'NXTT', 'CSLMR', 'FMBH', 'PROK', 'SHLT', 'BANR', 'FBIZ', 'OGI', 'XELA', 'AIRE', 'DJCO', 'NYMTL', 'RDHL', 'PDFS', 'MTEKW', 'NPAB', 'ONFO', 'OPINL', 'LVO', 'SIGA', 'TROO', 'CDIO', 'RILYP', 'SKYE', 'ODFL', 'PIRS', 'SMX', 'NVACW', 'KDP', 'CDNS', 'CYCN', 'PFMT', 'ATRO', 'AVTX', 'PLMJU', 'WINVW', 'WING', 'ZBRA', 'UPLD', 'FLGC', 'SWKS', 'AREB', 'DMLP', 'IZEA', 'ARAY', 'ZIONO', 'SQFTP', 'EOLS', 'NCPLW', 'FGBI', 'AGNCN', 'BLDE', 'DHAI', 'FEMY', 'GNTX', 'RBKB', 'PANL', 'SWVL', 'ACDC', 'AIRJW', 'BNIX', 'LENZ', 'TCBC', 'MGIH', 'CFSB', 'BENF', 'CGEN', 'FRST', 'PROF', 'GOEV', 'PWUPU', 'NAVI', 'OM', 'GLLIR', 'PCYO', 'AMPL', 'OCFCP', 'IMAB', 'OMCL', 'ARYD', 'PRLD', 'EXFY', 'HOLX', 'APXI', 'DOOO', 'BDRX', 'LPTH', 'MNDR', 'PFIE', 'MBINN', 'PLTK', 'EZPW', 'CHTR', 'PEV', 'AERT', 'DRRX', 'GNPX', 'CASS', 'FNWB', 'BFIN', 'BHF', 'GTEC', 'LNT', 'ACONW', 'LARK', 'ATOM', 'JFIN', 'MNRO', 'BCSAU', 'FAT', 'COEPW', 'HWH', 'LAES', 'CGBS', 'ONMDW', 'DWSN', 'ICLR', 'CPOP', 'NURO', 'SYRE', 'DRTS', 'UCBI', 'BCTX', 'NMTC', 'ARBE', 'CAN', 'BCDAW', 'FUND', 'NEOVW', 'SFBC', 'SUGP', 'XPON', 'LOOP', 'MXCT', 'ACMR', 'CISS', 'ATCOL', 'ONEW', 'VIRI', 'CBRGU', 'FGI', 'GVP', 'PLMJW', 'BRKHU', 'VRNT', 'AUTL', 'BIGC', 'CHDN', 'XENE', 'SSBK', 'FOXF', 'ALPP', 'APOG', 'BOWNU', 'CSLM', 'FFBC', 'LPSN', 'BRID', 'CRDF', 'MMV', 'PRZO', 'FATBW', 'LKQ', 'LXRX', 'FORA', 'USAP', 'XELB', 'VTVT', 'ESHAR', 'IPX', 'IMOS', 'BHFAP', 'OVID', 'RGP', 'USEA', 'ASTL', 'IVDAW', 'HAIAW', 'IOSP', 'IOVA', 'SBFG', 'LOT', 'COCO', 'GAUZ', 'ICU', 'CAAS', 'RIGL', 'SYT', 'OXLCN', 'FMAO', 'SNFCA', 'DAKT', 'EQ', 'GBDC', 'AMSWA', 'PHVS', 'SELF', 'DSGN', 'PDYNW', 'ATLO', 'EVGN', 'CCBG', 'GRWG', 'BTBDW', 'GBNY', 'HOVRW', 'IMNN', 'FBNC', 'BBIO', 'LLYVA', 'NRIM', 'CGNT', 'VWE', 'BYSI']

for ticker in tickers:
    try:
        df = import_data(ticker, '2024-01-01')
        findPivots(df, 5)
        findBreakouts(df, 100, 6, .001)
        # print(df.head(10))
        if (df.iloc[-1]['isBreakout'] == 1) & (df.iloc[-1]['Close'] > 5) & (df.iloc[-1]['STOCHk_14_3_3'] > 50) & (df.iloc[-1]['STOCHd_14_3_3'] > 50):
            with open('equity.txt', 'a') as f:
                f.write(f'Breakout Found at {ticker} \n')
    except Exception as e:
        print(e)
    

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

'NoneType' object is not iterable



[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['LDTCW']: YFInvalidPeriodError("%ticker%: Period 'max' is invalid, must be one of ['1d', '5d']")


'NoneType' object is not iterable


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

...

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

'NoneType' object is not iterable


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['HOVRW']: YFInvalidPeriodError("%ticker%: Period 'max' is invalid, must be one of ['1d', '5d']")
[*********************100%%**********************]  1 of 1 completed


'NoneType' object is not iterable


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [79]:
import oandapyV20.endpoints.orders as orders
import oandapyV20.endpoints.trades as trades
from oandapyV20 import API
from oanda_candles import Pair, Gran, CandleClient
import pandas_ta as ta
import time
import oandapyV20
import oandapyV20.endpoints.orders as orders
import oandapyV20.endpoints.accounts as accounts
from apscheduler.schedulers.blocking import BlockingScheduler

OANDA_API_KEY = 'afd9a0ce2842d7573c28373b4f1cd723-9c79d956e0ff25d200ae421ba1f70ffe'
OANDA_ACC_ID = '101-001-29478878-001'

In [80]:
def get_candles(n, instrument):
    client = CandleClient(OANDA_API_KEY, real=False)
    collector = client.get_collector(instrument, Gran.M5)
    candles = collector.grab(n)
    return candles

# candles = get_candles(3)
# for candle in candles:
#     print(float(str(candle.bid.o))>1)
#     print(candle)

In [81]:
def get_candles_frame(n, instrument):
    candles = get_candles(n, instrument)
    dfstream = pd.DataFrame(columns=['Open','Close','High','Low'])
    
    i=0
    for candle in candles:
        dfstream.loc[i, ['Open']] = float(str(candle.bid.o))
        dfstream.loc[i, ['Close']] = float(str(candle.bid.c))
        dfstream.loc[i, ['High']] = float(str(candle.bid.h))
        dfstream.loc[i, ['Low']] = float(str(candle.bid.l))
        i=i+1

    dfstream['Open'] = dfstream['Open'].astype(float)
    dfstream['Close'] = dfstream['Close'].astype(float)
    dfstream['High'] = dfstream['High'].astype(float)
    dfstream['Low'] = dfstream['Low'].astype(float)

    dfstream["ATR"] = ta.atr(dfstream.High, dfstream.Low, dfstream.Close, length=7)
    dfstream["EMA_fast"]=ta.ema(dfstream.Close, length=30)
    dfstream["EMA_slow"]=ta.ema(dfstream.Close, length=50)
    dfstream['RSI']=ta.rsi(dfstream.Close, length=10)
    my_bbands = ta.bbands(dfstream.Close, length=15, std=1.5)
    dfstream=dfstream.join(my_bbands)

    findPivots(dfstream, 10)
    findBreakouts(dfstream, 100, 11, .001)

    return dfstream    

In [82]:
def make_order(instrument, units, stop_loss, take_profit, order_type="MARKET"):
    order = {
        "order": {
            "units": str(units),
            "instrument": instrument,
            "type": order_type,
            "positionFill": "DEFAULT",
            "stopLossOnFill": {
                "price": stop_loss
            },
            "takeProfitOnFill": {
                "price": take_profit
            }
        }
    }
    client = oandapyV20.API(access_token=OANDA_API_KEY)
    r = orders.OrderCreate(accountID=OANDA_ACC_ID, data=order)
    try:
        response = client.request(r)
        print(response)
    except oandapyV20.exceptions.V20Error as e:
        print(f"Error placing order: {e}")

In [83]:
def get_equity():
    client = oandapyV20.API(access_token=OANDA_API_KEY)
    r = accounts.AccountSummary(accountID=OANDA_ACC_ID)
    response = client.request(r)
    equity = float(response['account']['balance'])
    return equity

In [84]:
def count_opened_trades():
    client = API(access_token=OANDA_API_KEY)
    r = trades.OpenTrades(accountID=OANDA_ACC_ID)
    client.request(r)
    return (r.response['trades'])

In [85]:
import requests

def get_instruments():
    results = []
    url = f'https://api-fxpractice.oanda.com/v3/accounts/{OANDA_ACC_ID}/instruments'
    headers = {
        'Authorization': f'Bearer {OANDA_API_KEY}',
    }
    response = requests.get(url, headers=headers)
    response.raise_for_status()  # Raise an exception if the request failed
    instrument_list = response.json()['instruments']
    for instrument in instrument_list:
        name = instrument['name']
        try:
            pair = getattr(Pair, name)
            print(pair)
            results.append(pair)
        except AttributeError:
            print(f'Pair for {name} not found in Pair class')
    return results

In [86]:
from backtesting import Strategy
from backtesting import Backtest
import seaborn as sns
import matplotlib.pyplot as plt

def backtest():
    stock = Pair.EUR_USD
    data = get_candles_frame(5000, stock)

    half = len(data) // 2
    data = data[half:]
    print(data[ data['isBreakout'] != 0])

    def SIGNAL():
            return data['isBreakout']

    class MyStrategy(Strategy):
        stop_loss_coef = 2.2
        take_profit_coef = 3.9

        def init(self):
            super().init()
            self.signal1 = self.I(SIGNAL)
            self.take_profit_next = None
        
        def next(self):
            super().next()
            stop_loss = self.stop_loss_coef * self.data.ATR[-1]
            price = 0.5
            
            # if a buy signal occurs, buy stock, then set a new take-profit value, then reset all stop-losses
            if (self.signal1 == 1):
                stop_loss_value = max(0, self.data.Close[-1] - stop_loss)
                self.buy( sl=stop_loss_value, size=price )
                # print('Buying at', self.data.Close[-1])

                self.take_profit_next = max(0, self.data.Close[-1] + (self.take_profit_coef*stop_loss) )

                for trade in self.trades:
                    trade.sl = stop_loss_value

            elif self.take_profit_next and (self.data.Close[-1] >= self.take_profit_next):
                if self.position and self.position.size > 0:
                    # print(f'Selling {int(self.position.size / 2) } shares at {self.data.Close[-1]}')
                    self.sell(size = int(self.position.size / 2) )

                    self.take_profit_next = max(0, self.data.Close[-1] + (self.take_profit_coef*stop_loss) )
                    stop_loss_value = max(0, self.data.Close[-1] - stop_loss)

                    for trade in self.trades:
                        trade.sl = stop_loss_value

    bt = Backtest(data, MyStrategy, cash=100, margin = 1, commission=.002)
    results = bt.run()
    print(results)
    print(results._trades)
    printCandlesticks(data)

    # stats, heatmap = bt.optimize(stop_loss_coef=[i/10 for i in range(10, 86)],
    #                     take_profit_coef=[i/10 for i in range(10, 46)],
    #                     maximize='Return [%]', max_tries=300,
    #                         random_state=0,
    #                         return_heatmap=True)
    # print(stats)
    # print(stats['_strategy'])
    # print(stats['_trades'])

    # heatmap_df = heatmap.unstack()
    # plt.figure(figsize=(14, 10))
    # sns.heatmap(heatmap_df, annot=True, cmap='viridis', fmt='.0f')
    # plt.show()
    # printCandlesticks(data)

# backtest()

In [87]:

import datetime


def manageAccount():
    instruments = get_instruments()
    stop_loss_coef = 2.2
    take_profit_coef = 3.9

    for instrument in instruments:
        data = get_candles_frame(200, instrument)

        curr_candle = data.iloc[-1]

        open_trades = [trade.get('instrument') for trade in count_opened_trades()]

        with (open('equity.txt', 'a')) as f:
            f.write(f'Breakout Value for {instrument}: {curr_candle["isBreakout"]}')
            f.write(f'Open Trades: {open_trades}')
        if (curr_candle['isBreakout'] == 1) and (not any(inst == instrument for inst in open_trades) ):
            curr_price = curr_candle['Close']

            total_equity = get_equity()
            units = int(total_equity / 2)
            stop_loss = max(0, curr_price - (stop_loss_coef * curr_candle['ATR']))
            take_profit = max(0, curr_price + (take_profit_coef * stop_loss_coef * curr_candle['ATR']))
            make_order(instrument, units, stop_loss, take_profit)
            with(open('equity.txt', 'a')) as f:
                f.write(f'Purchase Made: {units} units of {instrument} at {curr_price}')

    with (open('equity.txt', 'a')) as f:
            f.write('Ran the script at ' + str(datetime.datetime.now()) + '\n')

In [88]:
# scheduler = BlockingScheduler()
# scheduler.add_job(manageAccount, 'cron', day_of_week='mon-fri', hour='00-23', minute='1, 6, 11, 16, 21, 26, 31, 36, 41, 46, 51, 56',
#                   start_date='2023-12-08 12:00:00', timezone='America/Chicago')
# scheduler.start()

In [89]:
# manageAccount()